In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [3]:
sample = " if you want you"

In [4]:
# make dictionary
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{'y': 0, 't': 1, 'n': 2, 'a': 3, ' ': 4, 'i': 5, 'o': 6, 'u': 7, 'w': 8, 'f': 9}


In [5]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

In [6]:
# data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

In [7]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/var/folders/vc/qmhd_frs3436xpj2_4dshbtw0000gn/T/ipykernel_34957/1455435457.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1643987637853/work/torch/csrc/utils/tensor_new.cpp:201.)
  X = torch.FloatTensor(x_one_hot)


In [8]:
# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True)

In [9]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [10]:
# start training
for i in range(50):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  2.3217222690582275 prediction:  [[5 7 6 5 5 8 0 5 7 7 8 5 7 5 8]] true Y:  [[5, 9, 4, 0, 6, 7, 4, 8, 3, 2, 1, 4, 0, 6, 7]] prediction str:  iuoiiwyiuuwiuiw
1 loss:  2.038560152053833 prediction:  [[7 0 4 5 6 7 4 5 6 7 7 6 7 6 7]] true Y:  [[5, 9, 4, 0, 6, 7, 4, 8, 3, 2, 1, 4, 0, 6, 7]] prediction str:  uy iou iouuouou
2 loss:  1.8481826782226562 prediction:  [[0 0 4 4 0 0 4 8 0 7 4 0 0 6 7]] true Y:  [[5, 9, 4, 0, 6, 7, 4, 8, 3, 2, 1, 4, 0, 6, 7]] prediction str:  yy  yy wyu yyou
3 loss:  1.67733633518219 prediction:  [[0 0 4 0 0 0 4 8 6 7 1 0 0 6 7]] true Y:  [[5, 9, 4, 0, 6, 7, 4, 8, 3, 2, 1, 4, 0, 6, 7]] prediction str:  yy yyy woutyyou
4 loss:  1.5234806537628174 prediction:  [[0 0 4 0 6 7 4 5 6 2 1 4 0 6 7]] true Y:  [[5, 9, 4, 0, 6, 7, 4, 8, 3, 2, 1, 4, 0, 6, 7]] prediction str:  yy you iont you
5 loss:  1.4211878776550293 prediction:  [[0 0 4 0 6 7 4 5 3 2 1 0 0 6 7]] true Y:  [[5, 9, 4, 0, 6, 7, 4, 8, 3, 2, 1, 4, 0, 6, 7]] prediction str:  yy you iantyyou
6 loss:  1.32